# SQLite
В этом блокноте мы поговорим о *системе управления базами данных* (*СУБД*) **SQLite**.<br>
Преимуществом **SQLite** перед остальными *СУБД* заключается в том, что базы данных можно создавать и запускать прямо в **Google Colab**.

In [1]:
import sqlite3  # Модуль для взаимодействия с SQLite
# Попробуем подключиться к базе данных с названием test_db.db
try:
  conn = sqlite3.connect('test_db.db')
  print('Connected to MySQL database')
except Error as e:
    print(e)

Connected to MySQL database


Выполним SQL-запрос, который пробует удалить таблицу *tblCustomers*

In [2]:
try:
    # Пытаемся удалить таблицу. 
    conn.execute("DROP TABLE tblCustomers")
except:
    # Если ее не существует - игнорируем ошибку
    print('Удаляемая таблица не существует')
    pass

Удаляемая таблица не существует


Создадим таблицу *tblCustomers*:

In [3]:
conn.execute("CREATE TABLE tblCustomers (id INTEGER PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))")

Важно: чтобы первичный ключ генерировался автоматически, нужно указывать в типе данных именно **INTEGER**, а не **INT**.<br>
*Autoincrement* (*автоматическая генерация значений*, полезна для первичных ключей) в таком случае применяется автоматически.

Вставка новой записи в таблицу с помощью `conn.execute`, при этом для примера в качестве значений полей (name, address) передаем ('Igor', 'Pushkin street, Kolotushkin house')

In [4]:
conn.execute("INSERT INTO tblCustomers (name, address) VALUES ('Igor', 'Pushkin street, Kolotushkin house')")
conn.commit()  # commit сохраняет изменения в базе данных

Вставка новых записей в таблицу с помощью `conn.executemany`.
Вместо **?,?** будут подставлены соответствующие параметры из элементов списка. 

In [5]:
# Вставка нескольких строк
conn.executemany(
    "INSERT INTO tblCustomers (name, address) VALUES (?,?)",  # Вместо ?,? будут подставлены соответствующие параметры из элементов списка 
    [
        ('test2', '12345'),
        ('test3', '98765'),
    ]
)

Создаем курсор:

In [6]:
cursor = conn.cursor()

С помощью курсора выполняем команду извлечнеия всех записей из таблицы *tblCustomers*:

In [7]:
cursor.execute("SELECT * FROM tblCustomers")

С помощью `cursor.fetchone()` получаем 1 запись:

In [8]:
row = cursor.fetchone()  
conn.commit()

Выводим записи и, используя 'cursor', итеративно получаем новые:

In [9]:
while row:  
    print(row)  
    row = cursor.fetchone()

(1, 'Igor', 'Pushkin street, Kolotushkin house')
(2, 'test2', '12345')
(3, 'test3', '98765')


**Cursor** - это некая абстракция, которая имеет как совместные функции с объектом **Connection**, так и отличные от нее. **Cursor** здесь не то же самое, что и *Cursor* в базах данных. <br> Подробное описание представления можно найти, например, тут:
https://stackoverflow.com/questions/6318126/why-do-you-need-to-create-a-cursor-when-querying-a-sqlite-database <br>


Получить сразу все извлеченные из таблицы записи можно с помощью `cursor.fetchall()`:

In [10]:
cursor.execute("SELECT * FROM tblCustomers")

rows = cursor.fetchall()  # Получаем все записи  
print(rows)

[(1, 'Igor', 'Pushkin street, Kolotushkin house'), (2, 'test2', '12345'), (3, 'test3', '98765')]


Записи из таблицы можно извлекать "порциями" по указанному количеству записей (`chunk_size`):

In [11]:
def iter_rows(cursor, chunk_size):
    while True:
        # Получаем chunk_size записей за один раз
        rows = cursor.fetchmany(chunk_size)
        yield from rows
        if len(rows) < chunk_size:
            # Больше записей нет
            break

cursor.execute("SELECT * FROM tblCustomers")

rows = list(iter_rows(cursor, 2))
print(rows)

[(1, 'Igor', 'Pushkin street, Kolotushkin house'), (2, 'test2', '12345'), (3, 'test3', '98765')]


Изменяем запись и выводим результат:

In [12]:
cursor.execute("UPDATE tblCustomers SET name=? WHERE id=?", ('*EDIT*', 2))
conn.commit()

# Получаем все записи:
cursor.execute("SELECT * FROM tblCustomers")
rows = cursor.fetchall()
print(rows)

[(1, 'Igor', 'Pushkin street, Kolotushkin house'), (2, '*EDIT*', '12345'), (3, 'test3', '98765')]


Удаляем запись и выводи результат:

In [13]:
cursor.execute("DELETE FROM tblCustomers WHERE id=?", (1,))
conn.commit()

# Получаем все записи
cursor.execute("SELECT * FROM tblCustomers")
rows = cursor.fetchall()  
print(rows)
    

[(2, '*EDIT*', '12345'), (3, 'test3', '98765')]


закрываем курсор и подключение к базе данных:

In [14]:
cursor.close()
conn.close()

# Команды connection
С командами объекта **Connection** можно познакомиться, перейдя по [ссылке](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection).

## Команды курсора
С командами объекта **Cursor** можно познакомиться, перейдя по [ссылке](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor).